In [1]:
import tensorflow as tf
import numpy as np
from subprocess import call
import time, math
import h5py
from tqdm import tqdm

/home/mahasen/anaconda3/envs/intel_3.6_TF_1.8/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Setting dtype for tensorflow and numpy environments
DTYPE = tf.float64
DTYPE_np = np.float64
log_dir = '/mnt/dataDrive3/mahasen/tmp/funclearn' # Directory where we dump tensorboard log files

# Useful function for resetting logdir and tensorflow graph
def reset_tf():
    call(["rm","-rf",log_dir+'/'])
    tf.reset_default_graph()
    
    
# Define some functions for initialising layers with appropriate statistics    
def get_weights(shape,dtype):
    # Returns trainable weight variable, initialised from truncated (+\- 2std. dev. only) standard normal distribution
    return tf.Variable(tf.truncated_normal(shape, dtype=dtype),name='weights')

def get_biases(shape,dtype):
    # Returns trainable bias variable, initialised arbitrarily as a small constant
    return tf.Variable(tf.constant(0.01, shape=shape, dtype=dtype),name='biases')

def get_bn_offset(shape,dtype):
    # Returns trainable bias/offset variable for batch normalisation
    return tf.Variable(tf.truncated_normal(shape=shape, dtype=dtype),name='beta_offset')

def get_bn_scale(shape,dtype):
    # Returns trainable scale variable for batch normalisation
    return tf.Variable(tf.constant(1.0, shape=shape, dtype=dtype),name='gamma_scale')


def variable_summaries(var):
    # Attaches mean, stddev, max, min, and a histogram of an input var to a tensor
    # Useful for TensorBoard visualisation
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var,name='mean')
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)),name='stddev')
        
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
        
def batchnorm(logits, is_test, offset, scale, iteration):
    # Get summary statistics of this batch
    mean, variance    = tf.nn.moments(logits, [0],name='moments')
    
    # We'll use an exponential moving average over the training iterations during test time
    # This is a tool to do that
    exp_moving_avg    = tf.train.ExponentialMovingAverage(0.9999, iteration)
    update_moving_avg = exp_moving_avg.apply([mean, variance])
    
    # If this is the test, we use the m,v values we obtained from the exponential moving average 
    # over mean, variance that we obtained from training. otherwise use the batch mean, variance
    mean_cond        = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean, name='mean_cond')
    variance_cond    = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance, name='variance_cond')
    
    # This applies the following normalisation: x-> scale*(x-mean(x))/(variance_epsilon+std(x)) + offset
    logits_bn = tf.nn.batch_normalization(logits, mean_cond, variance_cond, offset, scale, variance_epsilon=1e-5,name='logits_batchnormed')
    
    return logits_bn, update_moving_avg

def get_layer_complete(input_tensor,input_dim, output_dim, layer_name, is_test, prob_keep, global_step, act_func=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = get_weights([input_dim, output_dim],DTYPE)
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = get_biases([output_dim],DTYPE)
            variable_summaries(biases)
        with tf.name_scope('batchnorm'):
            offset = get_bn_offset([output_dim], DTYPE)
            scale  = get_bn_scale([output_dim], DTYPE)
        
        logits = tf.add(tf.matmul(input_tensor, weights),biases,name='logits')
        tf.summary.histogram('logits', logits)
        logits_bn, update_moving_avg = batchnorm(logits, is_test, offset, scale, global_step)
        tf.summary.histogram('logits_batchNormed', logits_bn)
        activated = act_func(logits_bn, name='activation')
        dropped_out = tf.nn.dropout(activated,prob_keep,name='dropout')
        tf.summary.histogram('activations', activated)
        return dropped_out, update_moving_avg 
    
def func_deep_learner_complete(x,m,n,h,is_test,global_step,prob_keep,act_func=tf.nn.relu,num_layers=5):
    # x is the input variable, probably a TensorFlow placeholder object
    # m is the input dimension
    # n is the output dimension
    # h is the number of hidden neurons
    # act_func is the activation function to be applied
    # is_test is a flag is use to control batch normalisation behaviour during inferene on test data
    # iteration is the iteration counter used inside the training loop; required for batch normalisation
    
    bn_moving_avg_updates = []

    with tf.variable_scope('func_learner'):
        # 0th hidden layer
        hidden_layer, update_moving_avg = get_layer_complete(x,m,h,'hidden_layer_0',is_test,prob_keep,global_step)
        bn_moving_avg_updates.append(update_moving_avg)

        # Other hidden layers
        for i in range(num_layers-1):
            hidden_layer, update_moving_avg = get_layer_complete(hidden_layer,h,h,'hidden_layer_'+str(i+1),is_test,prob_keep,global_step)
            bn_moving_avg_updates.append(update_moving_avg)

        # Output layer
        weights1= tf.get_variable(name='output_layer_weights',
                              shape=[h,n],
                              initializer=tf.random_normal_initializer(),
                              dtype=DTYPE)

    return tf.matmul(hidden_layer,weights1), bn_moving_avg_updates

def train_RBM_complete(input_size=1,output_size=1,hidden_size=1,num_layers=5,model_func=None,unknown_func=None,
                  test_x=None, test_y = None, generate_training_samples_fun=None,
                  batch_size=25, max_steps = 1000, epochs =5,
                  initial_learning_rate = 0.02, decay_rate = 1/math.e, decay_steps=1000,
                  prob_keep = 0.8):
    
    with tf.Session() as sess:
            
        with tf.name_scope('input'):
            # Placeholder variables which will be fed data at train time
            x  = tf.placeholder(DTYPE,[None,input_size],name='x_input')
            y  = tf.placeholder(DTYPE,[None,output_size],name='y_input')
        
        with tf.name_scope('control_inputs'):
            global_step = tf.Variable(0, trainable=False,name='global_step')
            epoch_ind = tf.Variable(0,trainable=False,name='epoch_ind')
            increment_epoch_ind = tf.assign_add(epoch_ind,1,name='increment_epoch_ind')
            is_test = tf.contrib.eager.Variable(False, trainable=False,name='is_test')

        # Predictions of the NN
        y_pred, bn_moving_avg_updates = func_deep_learner_complete(x,input_size,output_size,hidden_size,is_test,global_step,1,num_layers=num_layers)

        # Should add dropout!

        with tf.name_scope('mean_squared_error'):
            mse = tf.losses.mean_squared_error(y,y_pred)
        tf.summary.scalar('mean_squared_error',mse)

        with tf.name_scope('train'):
            global_step_in_epoch = global_step - epoch_ind*max_steps
            learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step_in_epoch, decay_steps, decay_rate, staircase=True)
            tf.summary.scalar('learning_rate', learning_rate)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(mse, global_step = global_step)

        # Create merged summary object and file writers
        merged_summaries = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
        test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)
        
        # Initialise variables
        sess.run(tf.global_variables_initializer())
        
        # Train the model
        start_time = time.time()
        old_time = start_time
        for j in range(epochs):
            for i in range(max_steps):
                if i % 100 ==0:
                    # Check how our model performs against the test data
                    [summary, mse_val] = sess.run([merged_summaries,mse], feed_dict={x: test_x, y: test_y, is_test: True})
                    test_writer.add_summary(summary,i + j*max_steps)
                    curr_time = time.time()
                    print('Epoch %d, Step %04d, MSE: %4.3e,  LearnRate: %4.3e, Time taken : %4.3fs' % (sess.run(epoch_ind),i, mse_val, sess.run(learning_rate),curr_time-old_time))
                    old_time = curr_time
                else:
                    # Generate new sample data and train our model
                    train_x, train_y = generate_training_samples_fun(epoch_ind, global_step)
                    summary, _ = sess.run([merged_summaries, train_step], feed_dict={x: train_x, y: train_y, is_test: False})
                    sess.run(bn_moving_avg_updates, feed_dict={x: train_x, y: train_y, is_test: False})
                    train_writer.add_summary(summary,i+ j*max_steps)
            sess.run(increment_epoch_ind)
        train_writer.close()
        test_writer.close()     
        print('\nTotal time:\t %4.3fs' % (time.time()-start_time))

In [3]:
def func_deep_learner_arbshape(x,m,n,h,is_test,global_step,prob_keep,act_func=tf.nn.relu):
    # x is the input variable, probably a TensorFlow placeholder object
    # m is the input dimension
    # n is the output dimension
    # h is a LIST of hidden neurons
    # act_func is the activation function to be applied
    # is_test is a flag is use to control batch normalisation behaviour during inferene on test data
    # iteration is the iteration counter used inside the training loop; required for batch normalisation
    
    num_layers = len(h)
    
    bn_moving_avg_updates = []

    with tf.variable_scope('func_learner'):
        # 0th hidden layer
        hidden_layer, update_moving_avg = get_layer_complete(x,m,h[0],'hidden_layer_0',is_test,prob_keep,global_step)
        bn_moving_avg_updates.append(update_moving_avg)

        # Other hidden layers
        for i in range(num_layers-1):
            hidden_layer, update_moving_avg = get_layer_complete(hidden_layer,h[i],h[i+1],'hidden_layer_'+str(i+1),is_test,prob_keep,global_step)
            bn_moving_avg_updates.append(update_moving_avg)

        # Output layer
        weights1= tf.get_variable(name='output_layer_weights',
                              shape=[h[-1],n],
                              initializer=tf.random_normal_initializer(),
                              dtype=DTYPE)

    return tf.matmul(hidden_layer,weights1), bn_moving_avg_updates

In [4]:
def train_RBM_complete_mccv_inference(input_size=1,output_size=1,hidden_sizes=[5,5],x_data=None,y_data=None,
                  fname_model_in='/tmp/model.ckpt'):
    # Using montecarlo crossvalidation
    config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
    
    with tf.Session(config=config) as sess:

            
        with tf.name_scope('input'):
            # Placeholder variables which will be fed data at train time
            x  = tf.placeholder(DTYPE,[None,input_size],name='x_input')
            y  = tf.placeholder(DTYPE,[None,output_size],name='y_input')
        
        with tf.name_scope('control_inputs'):
            global_step = tf.Variable(0, trainable=False,name='global_step')
            epoch_ind = tf.Variable(0,trainable=False,name='epoch_ind')
            increment_epoch_ind = tf.assign_add(epoch_ind,1,name='increment_epoch_ind')
            is_test = tf.contrib.eager.Variable(False, trainable=False,name='is_test')

        # Predictions of the NN
        y_pred, bn_moving_avg_updates = func_deep_learner_arbshape(x,input_size,output_size,hidden_sizes,is_test,global_step,1)
        
        # Restore all the variables.
        saver = tf.train.Saver()
        saver.restore(sess,  fname_model_in)
        
        # Run inference
        predicted_values = sess.run(y_pred, feed_dict={x: x_data, is_test: False})
    return predicted_values

In [5]:
def run_model_ai_kup(acc,phase):
    n_samples  = acc.shape[0]
    seq_length = acc.shape[1]
    
    input_size  = seq_length
    output_size = 1
    
    hidden_sizes = [256,128,96,64,32,16,8]
    
    preds = train_RBM_complete_mccv_produceTrace(input_size=input_size,output_size=output_size,hidden_sizes=hidden_sizes,
                            x_data=acc,y_data=phase,fname_model_in='./model_SGD_Nesterov_2.ckpt')
    
    return preds

# Reconstruct transfer function
We will pass a series of cosine waves into the model. The amplitude will be fixed at the average of the original training data. We need to sweep frequency from 0.1mHz to 1kHz; phase from $-\frac{\pi}{2}-\frac{\pi}{2}$. $N$ discrete samples will be made on each of these ranges (logarithmically evenly spaced for frequency and uniformly spaced for phase.

For each frequency, we will construct a $(N,195)$ array which we will run inference on to produce a $(N,1)$ array. Concatenating all the arrays will produce a final $(N,N)$ array

In [6]:
N_freq = 10000
N_phase = 10001
dt= 3.200054168701172e-03 # modal dt in accelerometer data
amplitude = 4.784774538970486e-06 # std. dev. of accelerometer data

In [7]:
input_size = 195
t = np.arange(195)*dt

In [8]:
phases = np.linspace(-np.pi,np.pi,num=N_phase)
freqs = np.logspace(-4,2,num=N_freq) # 1e-4-100

In [9]:
# Define function that produces cosine wave
def make_cos_array(t,freq,phases):
    return np.array([np.cos(2*np.pi*freq*t+phase) for phase in phases])

In [10]:
reset_tf()

In [11]:
def transferFunction_sweep(t,freqs,phases):
    input_size = 195
    output_size= 1;
    hidden_sizes = [256,128,96,64,32,16,8]
    fname_model_in = './model_SGD_Nesterov_2.ckpt'
    # GPU has less memory than host
    config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )
    
    # Construct array for storage
    output = np.zeros([N_phase,N_freq],dtype=np.float64)
    
    with tf.Session(config=config) as sess:
        with tf.name_scope('input'):
            # Placeholder variables which will be fed data at train time
            x  = tf.placeholder(DTYPE,[None,input_size],name='x_input')
            y  = tf.placeholder(DTYPE,[None,output_size],name='y_input')
        
        with tf.name_scope('control_inputs'):
            global_step = tf.Variable(0, trainable=False,name='global_step')
            epoch_ind = tf.Variable(0,trainable=False,name='epoch_ind')
            increment_epoch_ind = tf.assign_add(epoch_ind,1,name='increment_epoch_ind')
            is_test = tf.contrib.eager.Variable(False, trainable=False,name='is_test')

        # Predictions of the NN
        y_pred, bn_moving_avg_updates = func_deep_learner_arbshape(x,input_size,output_size,hidden_sizes,is_test,global_step,1)
        
        # Restore all the variables.
        saver = tf.train.Saver()
        saver.restore(sess,  fname_model_in)
        
        # Run inference
        for i,freq in enumerate(tqdm(freqs)):
            cos_array = make_cos_array(t,freq,phases)
            predicted_values = sess.run(y_pred, feed_dict={x: cos_array, is_test: False})
            output[:,i] = predicted_values.reshape(len(predicted_values))
            
    return output

In [12]:
output = transferFunction_sweep(t,freqs,phases)

INFO:tensorflow:Restoring parameters from ./model_SGD_Nesterov_2.ckpt


100%|██████████| 10000/10000 [40:34<00:00,  4.11it/s]


In [13]:
# Write to file
fname_out = './campaign4a_kup_transferfun_10000.h5'
with h5py.File(fname_out,'w') as file:
    file.create_dataset('/phases',data=phases,dtype='float64')
    file.create_dataset('/freqs',data=freqs,dtype='float64')
    file.create_dataset('/t',data=t,dtype='float64')
    file.create_dataset('/output',data=output,dtype='float64')

In [26]:
import matplotlib.pyplot as plt

In [43]:
fig=plt.figure(figsize=(10, 10), dpi= 80, facecolor='w', edgecolor='k')
ax = imgplot = plt.imshow(output,extent=freqs)
plt.xscale('log')
plt.xlabel('Frequency',fontsize=16,fontweight='bold')
plt.ylabel('Phase',fontsize=16,fontweight='bold')
plt.show()

ValueError: too many values to unpack (expected 4)

In [26]:
np.min(freqs)

0.0001

In [24]:
1/(2*dt)

156.2473550886604